In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from scipy.stats import bernoulli
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [2]:
mushrooms_whole = pd.read_csv('mushroom.csv')
mushrooms = mushrooms_whole.sample(frac=0.8)
index_train = list(mushrooms.index)
mushrooms_test = mushrooms_whole.drop(labels=index_train,axis=0)
mushrooms = mushrooms.reset_index(drop=True)
n_samples = mushrooms.shape[0]
classes = mushrooms['class'].copy()
colnames = mushrooms.drop(['class'],axis=1).columns.copy()
mushrooms = StandardScaler().fit_transform(X=mushrooms.drop(['class'],axis=1).to_numpy())
mushrooms = pd.DataFrame(mushrooms)
mushrooms.columns = colnames
mushrooms['class'] = classes
mushrooms['label'] = np.ones(n_samples)*(-1)
for i in range(n_samples):
    random = bernoulli.rvs(p=3/4)
    if mushrooms.loc[i,'class'] == 1 and random == 0:
        mushrooms.loc[i,'label'] = 1

In [3]:
t=15
n_positives = mushrooms[mushrooms['label'] == 1].shape[0]
positive_instances = mushrooms[mushrooms['label'] == 1]
unlabelled_instances = mushrooms[mushrooms['label'] == -1]

In [4]:
df_results = mushrooms.copy()
df_results = mushrooms.drop(colnames,axis=1)

for i in tqdm(range(t)):
    u_t = unlabelled_instances.sample(n=n_positives)
    train_set = pd.concat([positive_instances, u_t])
    index_t = list(train_set.index)
    f_t = SVC(decision_function_shape='ovr').fit(X=train_set.to_numpy()[:,:-2],
                                          y=train_set['label'].to_numpy())
    to_test = mushrooms.copy()
    to_test = to_test.drop(labels = index_t)
    predictions_t = f_t.decision_function(X=to_test.to_numpy()[:,:-2])
    to_test[f'score_{i}'] = predictions_t
    to_test = to_test.drop(colnames, axis=1)
    to_test = to_test.drop(['class','label'], axis=1)
    df_results = df_results.merge(to_test, how='left',left_index=True,right_index=True)
    #df_pred = pd.DataFrame(to_test[f'score_{i}'])
    #df_results = df_results.merge(df_pred, how='left', left_index=True,
                                           #right_index=True)

100%|██████████| 15/15 [03:43<00:00, 14.90s/it]


In [5]:
"""
df_results = df_results.to_numpy()
for i in range(n_samples):
    df_results[i] = np.sign(df_results[i])


positives_train = 0
true_positives_train = 0   

for i in range(n_samples):
    if df_results[i] == 1:
        positives_train += 1
        if mushrooms.loc[mushrooms.index[i],'class'] == 1:
            true_positives_train += 1

precision_train = true_positives_train/positives_train
recall_train = true_positives_train/(mushrooms[mushrooms['class'] == 1].shape[0])
f_1_train = (2*precision_train*recall_train)/(precision_train+recall_train)
weird_train = (recall_train**2)/(positives_train/n_samples)
print('precision on train is : ', precision_train, '\n', 
      'recall on train is : ', recall_train, '\n', 
      'f_1 on train is : ', f_1_train, '\n', 
      'weird metric on train is : ', weird_train)



"""

"\ndf_results = df_results.to_numpy()\nfor i in range(n_samples):\n    df_results[i] = np.sign(df_results[i])\n\n\npositives_train = 0\ntrue_positives_train = 0   \n\nfor i in range(n_samples):\n    if df_results[i] == 1:\n        positives_train += 1\n        if mushrooms.loc[mushrooms.index[i],'class'] == 1:\n            true_positives_train += 1\n\nprecision_train = true_positives_train/positives_train\nrecall_train = true_positives_train/(mushrooms[mushrooms['class'] == 1].shape[0])\nf_1_train = (2*precision_train*recall_train)/(precision_train+recall_train)\nweird_train = (recall_train**2)/(positives_train/n_samples)\nprint('precision on train is : ', precision_train, '\n', \n      'recall on train is : ', recall_train, '\n', \n      'f_1 on train is : ', f_1_train, '\n', \n      'weird metric on train is : ', weird_train)\n\n\n\n"

In [6]:
mushrooms_bis = mushrooms.copy()
df_results_gen = mushrooms_bis.merge(df_results, how='left', left_index=True, right_index=True)

In [7]:
df_results_gen['average'] = df_results.mean(axis=1)

In [8]:
df_results_gen = df_results_gen.drop(colnames,axis=1)

In [9]:
positives_train = 0
true_positives_train = 0

for i in df_results_gen.index:
    df_results_gen.loc[i,'average'] = np.sign(df_results_gen.loc[i,'average'])

for i in df_results_gen.index:
    if df_results_gen.loc[i,'average'] == 1:
        positives_train += 1
        if mushrooms.loc[i,'class'] == 1:
            true_positives_train += 1

precision = true_positives_train/positives_train
recall = true_positives_train/n_positives
f_1_train = (2*precision*recall)/(precision+recall)
weird_thing = (recall**2)/(positives_train/n_samples)

In [10]:
print('the precision on train set is : ', precision, '\n', 
      'recall on train set : ', recall, '\n', 
      'f_1 in train set : ', f_1_train, '\n',
      'weird on train : ', weird_thing)

the precision on train set is :  0.3791427046896306 
 recall on train set :  1.3166563722462425 
 f_1 in train set :  0.5887497698398084 
 weird on train :  4.4429578030107795


In [7]:
mushrooms_ter = mushrooms.drop(list(positive_instances.index),axis=0)
mushrooms_ter = mushrooms_ter.merge(df_results, how='left', right_index=True, left_index=True)

In [8]:
mushrooms_ter = mushrooms_ter.drop(colnames, axis=1)
mushrooms_ter

,class_x,label_x,class_y,label_y,score_0,score_1,score_2,score_3,score_4,score_5,score_6,score_7,score_8,score_9,score_10,score_11,score_12,score_13,score_14
0,1,-1.0,1,-1.0,0.926031,0.884438,1.117645,NaN,NaN,0.991759,0.950093,NaN,1.009667,1.246392,1.178786,1.020742,NaN,NaN,0.927808
1,0,-1.0,0,-1.0,-0.928822,-1.011968,-1.111508,-1.341211,-0.859780,-1.069923,-0.964332,-1.202346,-1.012885,-0.946921,-0.707844,-0.795110,-0.903130,NaN,-1.132032
2,1,-1.0,1,-1.0,1.007902,1.028418,1.024189,1.052454,1.023899,1.049333,1.032243,1.058842,1.074147,1.075870,1.075067,NaN,1.026874,1.094117,0.932907
3,0,-1.0,0,-1.0,-1.383310,-1.350355,-1.447157,-1.421978,-1.261333,-1.330694,-1.209569,-1.416679,-1.227028,-1.310594,-1.308627,-1.421635,-1.389287,-1.416524,-1.359699
4,1,-1.0,1,-1.0,1.118121,1.141395,1.197433,1.051788,1.093133,1.106034,NaN,NaN,1.180181,1.213176,1.209662,NaN,1.243210,1.131812,1.046672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43223,1,-1.0,1,-1.0,0.982906,1.038633,1.067333,1.008306,1.059129,1.124048,0.981012,1.041912,0.993102,1.050467,1.026251,NaN,1.000626,1.146403,0.995380
43224,1,-1.0,1,-1.0,1.066268,NaN,1.068633,1.078998,1.077724,1.106430,1.068600,1.082745,0.951774,0.926142,1.044550,1.089772,1.072661,1.053251,NaN
43225,1,-1.0,1,-1.0,0.888062,0.947082,NaN,0.958194,0.935363,0.954067,0.931549,0.926425,0.962595,0.929875,0.872241,0.996761,0.957596,0.982591,NaN
43226,1,-1.0,1,-1.0,1.054555,1.056716,1.099307,1.057699,1.073272,1.092033,1.032395,1.024547,1.067860,1.106494,1.021519,NaN,1.050859,1.084901,1.077275


In [5]:
mushrooms.drop(colnames,axis=1)

,class,label
0,0,-1.0
1,0,-1.0
2,1,-1.0
3,1,-1.0
4,1,1.0
...,...,...
43223,1,-1.0
43224,1,-1.0
43225,1,1.0
43226,1,-1.0
